In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W27` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W28` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W29`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W30`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W31`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W32`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W33`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W34`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W35`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W36`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W37`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W38`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W39`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
98252,2020-07-26 12:09:36+00:00,2020-07-26 12:14:53+00:00,Production,0018 of 0083,C2,None,None,KL19,Monday Assembly,Sunday Morning,Siti Rahmah SiregarSiti Rahmah Siregar,3,5.28,2020-31
98253,2020-07-26 13:36:04+00:00,2020-07-26 13:42:54+00:00,Production,0029 of 0083,C2,None,None,KL19,Monday Assembly,Sunday Morning,Siti Rahmah SiregarSiti Rahmah Siregar,3,6.83,2020-31
98254,2020-07-26 10:27:50+00:00,2020-07-26 10:32:43+00:00,Production,0009 of 0083,C2,None,None,KL19,Monday Assembly,Sunday Morning,Siti Rahmah SiregarSiti Rahmah Siregar,3,4.88,2020-31
98255,2020-07-26 10:10:33+00:00,2020-07-26 10:17:06+00:00,Production,0007 of 0083,C2,None,None,KL19,Monday Assembly,Sunday Morning,Siti Rahmah SiregarSiti Rahmah Siregar,3,6.55,2020-31
98256,2020-07-26 12:14:53+00:00,2020-07-26 12:20:08+00:00,Production,0019 of 0083,C2,None,None,KL19,Monday Assembly,Sunday Morning,Siti Rahmah SiregarSiti Rahmah Siregar,3,5.25,2020-31


In [5]:
data_in_use.shape

(98257, 14)